# Using the GridR's core grid resampling API

This guide shows how to use the core `array_grid_resampling` GridR's feature.

## Setting things up

In [ ]:
import os
import sys

import numpy as np
from notebook_utils import plot_im

sys.path.insert(0, "/".join(["..","python"]))

IN_DOC_BUILD = os.environ.get("DOC_BUILD", "0") == "1"
if not IN_DOC_BUILD:
    from bokeh.io import output_notebook # enables plot interface in J notebook
    output_notebook()

First we have to import the `array_grid_resampling` method.

We also import the well known mandrill raster image.

In [ ]:
from gridr.core.grid.grid_resampling import array_grid_resampling
from gridr.misc.mandrill import mandrill

Please note the mandrill image has 3 color's channels.

Here we limit the image to the first channel and convert its type to 64 bits floating points.

In [ ]:
image = mandrill[0]
grid_dtype = np.float64
data_in_dtype = np.float64
data_out_dtype = np.float64

array_in = image.astype(data_in_dtype)

## Identity grid transform

We start by defining the idendity transform grid. 

That's not really impressive but we will go further to illustrate some variants (windowing, zooming, ...)

Let's first create the idendity transformation grid.

In [ ]:
# create identity grid
if image.ndim == 2:
    x = np.arange(0, image.shape[0], dtype=grid_dtype)
    y = np.arange(0, image.shape[1], dtype=grid_dtype)
xx, yy = np.meshgrid(x, y)

### Simple idendity transform

The `array_grid_resampling` method provides the `array_out` argument in order to use a preallocated C-contiguous array as output. It is quite convenient when implementing a tiling chain that uses that method.

For now, we choose to keep it simple : by setting `array_out` to `None` we let the method allocate the output buffer for us.

We have defined the rows and columns grids to be at full resolution, ie. there is one coordinate in the grids for each pixel in the input image. To stay at that resolution we set `grid_resolution` to 1 in both directions.

The `array_grid_resampling` method returns a Tuple of 2 numpy arrays. For now we just focus on the first array : it corresponds to the allocated output buffer, meaning that's our interpolation result !

In [ ]:
# Lets call the grid resampling
array_out_id, _ = array_grid_resampling(
        array_in=image.astype(data_in_dtype),
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

In [ ]:
# Show the window
plot_im(array_out_id, win_rect=None, prefix='001')

In the following test we compare input with output. It must succeed !

In [ ]:
np.testing.assert_almost_equal(array_in, array_out_id, decimal=10)

### Apply a window

The `array_grid_resampling` method provides the `win` parameter in order to set a window to limit the region to compute.

The indices used in the `win` definition refers to indices considering the full resolution output.

Below, we first define a window centered on the mandrill's left eye. Since we are considering an identity grid, the grid coordinates align with the input array's image coordinates

In [ ]:
win_center = np.array((58, 175)) # left eye
win_shape =  np.array((100, 100))
win = np.array((win_center - win_shape//2, win_center - win_shape//2 + win_shape - 1)).T

In [ ]:
# Show the window in the full image
plot_im(array_out_id, win_rect=win, prefix='002')

In [ ]:
array_out_id_win_1_1, _ = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=win,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

In [ ]:
# Let's show it
plot_im(array_out_id_win_1_1, None, prefix='003')

In [ ]:
expected_array_out = array_out_id[win[0][0]:win[0][1]+1, win[1][0]:win[1][1]+1]
np.testing.assert_almost_equal(expected_array_out, array_out_id_win_1_1, decimal=10)

### Playing with resolution to apply a zoom

The `array_grid_resampling` method uses the  `grid_resolution` parameter to insert additional nodes between existing grid nodes. This insertion is achieved through bilinear interpolation, with the oversampling factors for both rows and columns specified in the grid_resolution parameter.

While the primary purpose of the `grid_resolution` is to control computation time and grid storage, in this case, we use it to perform a zoom operation, increasing the row resolution by a factor of 2 and the column resolution by a factor of 5.

At the same time, we want to maintain the same input region as before. To achieve this, the window definition must be adjusted to accommodate the new full-resolution output.

In [ ]:
# Set resolution to the target zoom factors
resolution = (2, 3)

# The full resolution grid has (Nrow-1)*resolution_row + 1 rows (apply same logic on column)
full_output_shape = (np.asarray(array_in.shape) - 1)*resolution + 1

# setting output window to only consider a region of interest
# here we construct a window centered on the left eye in the full resolution output grid
win_center = np.array((58, 175)) * np.array(resolution)
win_shape =  np.array((100, 100)) * np.array(resolution)

win = np.array((win_center - win_shape//2, win_center - win_shape//2 + win_shape - 1)).T

# Lets call the grid resampling
array_out_id_zoom_2_5, _ = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=resolution,
        array_out=None,
        win=win,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

In [ ]:
# Let's show it
plot_im(array_out_id_zoom_2_5, None, prefix='004')

## Geometry transformation examples

In that part we provide some examples of simple non identity transformation.

### Translation transform

Let's begin with a simple translation on the column coordinates by shifting the image to the left.

In [ ]:
# First create identity grid
if image.ndim == 2:
    x = np.arange(0, image.shape[0], dtype=grid_dtype)
    y = np.arange(0, image.shape[1], dtype=grid_dtype)
xx, yy = np.meshgrid(x, y)
xx += 50.5

# Lets call the grid resampling
array_out_translation_left, _ = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

In [ ]:
plot_im(array_out_translation_left, None, prefix='005')

In [ ]:
array_out_translation_left[:, -53:]

As you can see, the right strip border is filled with `NaN` values. It is due to the fact that the grid target coordinates went outside of the image's definition domain.

Please note that the filling with `NaN` is not the default behaviour and it occured because we set the `nodata_out` paramater to `None`.

Let's replay it by setting `nodata_out` to its default value, ie 0.

As you will see, the right strip border is now filled with zeros.

In [ ]:
array_out_translation_left, array_out_translation_left_mask = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=True,
        nodata_out=0.,
        )
array_out_translation_left[:, -53:]

Let's show the output validity mask

In [ ]:
array_out_translation_left_mask[:, -53:]

In [ ]:
plot_im(array_out_translation_left_mask, None, prefix='005-mask-out')

### Rotation transform

Here we define a grid in order to apply a 45 degree rotation.

In order to have the full image rotated please note that we have to extend the grid size.

In [ ]:
center = np.asarray(array_in.shape) // 2
theta = np.pi/4.
# we know the image is square - here we extend the output grid in order to cover
# the full image
ext = image.shape[0] * (2**0.5 - 1) // 2

# First create identity grid
if image.ndim == 2:
    x = np.arange(0, image.shape[0] + 2 * ext, dtype=grid_dtype) - ext
    y = np.arange(0, image.shape[1] + 2 * ext, dtype=grid_dtype) - ext
xx, yy = np.meshgrid(x , y)
# Apply the rotation
xrot = (xx - center[1]) * np.cos(theta) - (yy - center[0]) * np.sin(theta) + center[1]
yrot = (xx - center[1]) * np.sin(theta) + (yy - center[0]) * np.cos(theta) + center[0]

As a demonstration purpose we here preallocate the array_out buffer.

In [ ]:
# We have to init the out buffer with the right shape
array_out = np.zeros(xx.shape, dtype=data_out_dtype)

# Lets call the grid resampling
array_out, _ = array_grid_resampling(
        array_in=array_in,
        grid_row=yrot,
        grid_col=xrot,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_mask=None,
        array_out_mask=None,
        nodata_out=None,
        )

In [ ]:
plot_im(array_out, None, prefix='006')

## Working with masks

In this part we will see how to use masking.

The `array_grid_resampling` method provides several kind of masking.

### Masking with a raster mask

Here we use an 8 bit raster having the same shape of the grids in order to devalidate points in the grid.

To keep it simple we will demonstrate using an identity transform.

In [ ]:
# First create identity grid
if image.ndim == 2:
    x = np.arange(0, image.shape[0], dtype=grid_dtype)
    y = np.arange(0, image.shape[1], dtype=grid_dtype)
xx, yy = np.meshgrid(x, y)

We define the mask as following :
* the first 10 lines and last 20 lines are masked
* the first 30 columns and last 40 columns are masked
* a 80x80 window positionned at the grid center are masked

Please note the mask applies to the grid and not to the input image.

By convention, the mask is considered binary. A value of 0 corresponds to a masked cell, a value of 1 corresponds to a valid cell.

In [ ]:
# setting a mask to devalidate margins and center
grid_mask_valid_value = 1
grid_mask_invalid_value = 0
# the final multiplication here is not necessary but we keep it to show the mechanic
grid_mask = np.ones(xx.shape, dtype=np.uint8) * grid_mask_valid_value
# masked pixels are positiv ones
grid_mask[0:10] = grid_mask_invalid_value 
grid_mask[-20:] = grid_mask_invalid_value 
grid_mask[:,0:30] = grid_mask_invalid_value 
grid_mask[:,-40:] = grid_mask_invalid_value
# devalidate center
grid_mask[image.shape[0]//2-40:image.shape[0]//2+40, image.shape[1]//2-40:image.shape[1]//2+40] = grid_mask_invalid_value

The `nodata_out` is used to set the output value affected by the masking operation.

In [ ]:
# Lets call the grid resampling
array_out, _ = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_mask=grid_mask,
        grid_mask_valid_value=grid_mask_valid_value,
        array_out_mask=None,
        nodata_out=0,
        )

In [ ]:
plot_im(array_out, prefix='007')

### Masking with an inplaced grid nodata value

In this case, we avoid providing an additional mask, primarily to limit memory usage.

The `array_grid_resampling` method allows specifying a `grid_nodata` parameter, which is used to invalidate grid cells.

Cells in the grid whose values match this nodata value (within a fixed absolute tolerance of 1e-5) are treated as masked.

Providing a non-`None` value for `grid_nodata` activates the masking mechanism.

Note that it is not necessary to assign the nodata value to both grid coordinate arrays (row and column); setting it in either one is sufficient for the corresponding cell to be considered masked.

In [ ]:
# Set a nodata value
grid_nodata = -999.
# Apply it to a window of the grid
yy[50:200,100:350] = grid_nodata

In [ ]:
# Lets call the grid resampling
# Spoiler alert: this will fail !
try:
    array_out, _ = array_grid_resampling(
            array_in=array_in,
            grid_row=yy,
            grid_col=xx,
            grid_resolution=(1,1),
            array_out=None,
            win=None,
            array_in_mask=None,
            grid_nodata=grid_nodata,
            grid_mask=grid_mask,
            grid_mask_valid_value=grid_mask_valid_value,
            array_out_mask=None,
            nodata_out=0,
            )
except ValueError as e:
    display(e)

The call failed, and the ValueError exception provides the reason: both masking modes, grid_mask and grid_nodata, cannot be used simultaneously.

In [ ]:
# Lets remove the grid_mask parameter's definition
array_out, _ = array_grid_resampling(
        array_in=array_in,
        grid_row=yy,
        grid_col=xx,
        grid_resolution=(1,1),
        array_out=None,
        win=None,
        array_in_mask=None,
        grid_nodata=grid_nodata,
        grid_mask=None,
        grid_mask_valid_value=None,
        array_out_mask=None,
        nodata_out=0,
        )

In [ ]:
plot_im(array_out, prefix='008')